# Getting Started

## Starting sc3nb

In [1]:
import sc3nb as scn

To startup sc3nb (sclang, scsynth, and a python OSC server) use `startup` which will return a `SC` instance which is the central SuperCollider Interface

In [2]:
sc = scn.startup()  # see Configuration of sc3nb startup below for more details

<IPython.core.display.Javascript object>

Starting sclang process... 

Done.
Registering OSC /return callback in sclang... Done.
Loading default sc3nb SynthDefs... Done.
Booting SuperCollider Server... 

Done.


You can produce a test sound with `blip`, which should relax any tensions whether the server is up and running. This sound should be played by the default server start.

In [3]:
sc.server.blip()

`sc` provides you the interfaces for

* `scsynth` via `sc.server`
* `sclang` via `sc.lang`

**Configuration of sc3nb startup**:

- Specifying the excutable location
  - We try to find the sclang and scsynth executables in the `$PATH` environment variable and also in the default installation directories
      - On macOS they reside in `/Applications/SuperCollider.app/Contents` in the folders MacOS and Resources. <br>
    To add these paths to your `$PATH`, simply add to your ~/.profile, e.g. (please adapt to your installation): <br>
    `PATH=$PATH:/Applications/SuperCollider.app/Contents/MacOS:/Applications/SuperCollider.app/Contents/Resources`  
      - On Windows they reside in your Installation folder f.e `C:\Program Files\SuperCollider-3.x.x`

  - If the executables are not found, you can specify them with `sclang_path` / `scsynth_path` e.g. `sclang_path="/path/to/sclang-containing-dir/sclang"`
- You could also only use **scsynth** only and don't start **sclang** with `start_sclang=False`
- See `help(scn.startup)` for all options

In [4]:
# help(scn.startup)

## Basic examples

### Use Python implementations to control the SuperCollider audio server

Create and control SuperCollider Synths

In [5]:
syn = scn.Synth()
syn

<Synth(20001) 'default' s {}>

In [6]:
syn.freq = 800
syn

<Synth(20001) 'default' s {'freq': 800}>

In [7]:
syn.free()  # or use the ctrl-. (cmd-.) shortcut in Jupyter notebooks
syn

<Synth(20001) 'default' f {'freq': 800}>

Send SynthDefs with python code injection

In [8]:
synth_dur = 0.5  # you can use python variable values in SynthDefs
synth_def = scn.SynthDef('random', """{ |out|
Out.ar(out, SinOsc.ar(Rand(400, 800), 0, 0.2) * Line.kr(1, 0, ^synth_dur, doneAction: Done.freeSelf))
}""")
synth_def.add()

scn.Synth("random")

<Synth(20002) 'random' s {}>

Load a file as Buffer and play it

In [9]:
buf = scn.Buffer().read("./media/blip.wav")
buf

<Buffer(128) on ('127.0.0.1', 57110): 2 x 43008 @ 44100 Hz = 0.975s allocated using mode 'file'>

In [10]:
buf.play()

<Synth(20003) 'sc3nb_playbuf_128' s {}>

In [11]:
buffer_data = buf.to_array()
print(buffer_data.shape)
buffer_data

(43008, 2)


array([[-0.00189209, -0.00189209],
       [-0.00430298, -0.00430298],
       [-0.00708008, -0.00708008],
       ...,
       [ 0.19314575,  0.19314575],
       [ 0.18963623,  0.18963623],
       [ 0.18585205,  0.18585205]])

### Use OSC to control the SuperCollider audio server

Send OSC Messages with [SuperCollider Commands](https://doc.sccode.org/Reference/Server-Command-Reference.html) and receive replies

In [12]:
sc.server.msg("/status")

(1,
 9,
 1,
 9,
 117,
 0.25753745436668396,
 0.43315771222114563,
 44100.0,
 43577.184415584416)

In [13]:
sc.server.status()

ServerStatus(num_ugens=9, num_synths=1, num_groups=9, num_synthdefs=117, avg_cpu=0.25831735134124756, peak_cpu=0.43315771222114563, nominal_sr=44100.0, actual_sr=43548.905905256324)

Create OSC Bundles using SuperCollider Commands directly

In [14]:
with sc.server.bundler() as bundler:
    bundler.add(0.2, "/s_new", ["s1", -1, 0, 0])
    bundler.add(0.5, "/s_new", ["s1", -1, 0, 0, "freq", 800, "dur", 0.1])
bundler.messages()

{0.2: [<OSCMessage("/s_new", ['s1', -1, 0, 0])>],
 0.5: [<OSCMessage("/s_new", ['s1', -1, 0, 0, 'freq', 800, 'dur', 0.10000000149011612])>]}

or create OSC Bundles with the high level Python implementations

In [15]:
with sc.server.bundler() as bundler:
    bundler.wait(0.2)
    scn.Synth("s1")
    bundler.wait(0.3)
    scn.Synth("s1", {"freq": 800, "dur": 0.1})
bundler.messages()

{0.2: [<OSCMessage("/s_new", ['s1', 20004, 0, 67108865])>],
 0.5: [<OSCMessage("/s_new", ['s1', 20005, 0, 67108865, 'freq', 800, 'dur', 0.10000000149011612])>]}

### Use the SuperCollider Language from Python

Execute SuperCollider Language Code

In [16]:
breakfast = "eggs"
sc.lang.cmd('^breakfast.scramble;')

-> egsg


or via the **%sc** IPython magic

In [17]:
%sc ^breakfast.scramble

-> eggs


Get results from SuperCollider Language in python with `cmdg` or **%scg**

In [18]:
x = 5
value = %scg (1..^x)
print(f"received {value} with type {type(value)}")

-> [ 1, 2, 3, 4, 5 ]
received [1, 2, 3, 4, 5] with type <class 'list'>


### Combined usage

create a Synth with sclang and get the nodeid of the Synth via the **%scg** IPython magic

In [19]:
nodeid = %scg x = Synth("default"); x.nodeID;

-> 1000


create the corresponding Python Synth instance

In [20]:
synth = scn.Synth("default", nodeid=nodeid, new=False)
synth

<Synth(1000) 'default'  {}>

Inspect and control the Synth from Python

In [21]:
synth.synth_desc

{'out': SynthArgument(name='out', rate='scalar', default=0.0),
 'freq': SynthArgument(name='freq', rate='control', default=440.0),
 'amp': SynthArgument(name='amp', rate='control', default=0.10000000149011612),
 'pan': SynthArgument(name='pan', rate='control', default=0.0),
 'gate': SynthArgument(name='gate', rate='control', default=1.0)}

In [22]:
synth.freq

440.0

In [23]:
synth.freq *= 2
synth.freq

880.0

In [24]:
synth

<Synth(1000) 'default'  {'freq': 880.0}>

send a OSC Message to free the Synth

In [25]:
sc.server.msg("/n_free", nodeid)

In [26]:
synth

<Synth(1000) 'default'  {'freq': 880.0}>

To shut down the server and sclang subprocesses

In [27]:
sc.exit()  

## Further information and advanced examples

For more details on the specific parts please refer to the corresponding sections of the User Guide.

- [SuperCollider Language **sclang**](./sclang-examples.ipynb)

- Usage of the SuperCollider Objects python interface
  - [Server](./supercollider-objects/server-examples.ipynb)
  - [Nodes (Synth and Group)](./supercollider-objects/node-examples.ipynb)
  - [SynthDef](./supercollider-objects/synthdef-examples.ipynb)
  - [Buffer](./supercollider-objects/buffer-examples.ipynb)
  - [Bus](./supercollider-objects/bus-examples.ipynb)
  - [Recorder](./supercollider-objects/recorder-examples.ipynb)


- Usage of common SuperCollider functions [Helpers](./helper-examples.ipynb)
- Usage of OSC in SuperCollider: [OSC Communication](./osc-communication-example.ipynb)